In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from utilities import lines2wv_model

In [ ]:
# 분석용 데이터프레임/사전준비 w2v 모델 준비
if os.path.isfile('../web2db/saved/df_summary.pickle'):
    df_summary = pd.read_pickle("../web2db/saved/df_summary.pickle")
else: df_summary = pd.DataFrame()
print(df_summary.info())

s_no =100000

# model = Word2Vec.load("C:/Users/hcjeo/VSCodeProjects/web2db/wvec_2021-07-11-15-58-56.model")
wvec = lines2wv_model.Lines2Model()
text_data = list(df_summary['items'])[:s_no]
model = wvec.run(text_data)

In [ ]:
# 문장 임베딩: 단어벡터 평균
word_vectors = model.wv
sentence_vector_list = []
for i in df_summary['items'][0:s_no]:
    word_vectors_list_for_an_item = np.empty((100, 0), float)
    for v in i.split(' '):
        if v in word_vectors.key_to_index:
            word_vectors_list_for_an_item = np.append(word_vectors_list_for_an_item, word_vectors[v].reshape(100, 1), axis = 1)
        else:
            word_vectors_list_for_an_item = np.append(word_vectors_list_for_an_item, word_vectors['판결'].reshape(100, 1), axis = 1)
    word_vectors_list_for_an_item_mean = word_vectors_list_for_an_item.mean(axis=1)
    sentence_vector_list.append(list(word_vectors_list_for_an_item_mean))
print(np.array(sentence_vector_list).shape)

In [ ]:
# 문장 임베딩: TF IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(text_data)
sentence = [text_data[3]] 
print(sentence)
print(tfidf_vectorizer.transform(sentence).toarray())
sentence_vector_list_ = np.array([tfidf_vectorizer.transform([x]).toarray() for x in text_data])
print(sentence_vector_list_.shape)
sentence_vector_list_ = sentence_vector_list_.reshape(10000, 42301)
print(np.array(sentence_vector_list_).shape)

In [ ]:
# KMEANS 군집 wv mean for K neighbor
k = 5
sentence_vector_list = np.array(sentence_vector_list)
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(sentence_vector_list)

labels = list(y_pred)
item_ =  list(df_summary['items'][0:s_no])
series = pd.Series(item_, index = labels)
df = series.to_frame().reset_index()
print(df.head(10))
# df.to_csv("df_500_kmeans.csv", encoding='utf-8-sig')

t, te, tl, tel = train_test_split(np.array(sentence_vector_list, dtype = object), np.array(labels, dtype= np.int32), test_size = 0.25, random_state=42)
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(t, tl)
predict_label = knn.predict(te)
print('test accuracy {:.2f}'.format(np.mean(predict_label == tel)))

In [ ]:
# KMEANS 군집 tf idf for K neighbor
k = 5
sentence_vector_list = sentence_vector_list_
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(sentence_vector_list)

labels = list(y_pred)
item_ =  list(df_summary['items'][0:s_no])
series = pd.Series(item_, index = labels)
df = series.to_frame().reset_index()
print(df.head(10))
# df.to_csv("df_500_kmeans.csv", encoding='utf-8-sig')

t, te, tl, tel = train_test_split(np.array(sentence_vector_list, dtype = object), np.array(labels, dtype= np.int32), test_size = 0.25, random_state=42)
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(t, tl)
predict_label = knn.predict(te)
print('test accuracy {:.2f}'.format(np.mean(predict_label == tel)))

In [ ]:
# ndarray shape 변환 연습
k = [
        [
            [
                [
                    1, 2, 3
                ],
                [
                    4, 5, 6
                ]
            ]
        ],

        [
            [
                [
                    11, 21, 31
                ],
                [
                    41, 51, 61
                ]
            ]
        ]
    ]
tt = np.array(k)
print(tt.shape)
ttt = tt.reshape(2, 2, 3)
print(ttt.shape)
print(ttt)